In [1]:
from opentrons import execute, types

In [2]:
metadata = {
    'protocolName': 'Nanopore LSK109(3) Adapter Ligation_miniON',
    'author': 'Seong-Kun Bak <tjdrns27@kribb.re.kr>',
    'apiLevel': '2.11'
}

In [4]:
protocol = execute.get_protocol_api("2.11")

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


In [200]:
protocol.home()

In [6]:
module_thermocycler = protocol.load_module("thermocycler")
module_magnetic = protocol.load_module('magnetic module gen2', '4')
#module_thermocycler = protocol.load_module("thermocycler", "7")

In [7]:
import json
with open("./labware/bioneer_96_tuberack_200ul.json") as labware_file:
    bioneer_tube = json.load(labware_file)
with open("./labware/biorad_96_tuberack_200ul.json") as labware_file:
    biorad_tube = json.load(labware_file)

In [8]:
enzyme_rack = protocol.load_labware("opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap", 1)
falcon_rack = protocol.load_labware("opentrons_6_tuberack_falcon_50ml_conical", 2)
dna_plate = protocol.load_labware_from_definition(bioneer_tube, 5)
mag_plate = module_magnetic.load_labware("biorad_96_tuberack_200ul", 4)
trash = protocol.loaded_labwares[12]["A1"]

tiprack_20_sin = protocol.load_labware("opentrons_96_tiprack_20ul", 3)
tiprack_300_sin = protocol.load_labware("opentrons_96_tiprack_300ul", 6)

This labware ({}) is not explicitly compatible with the Magnetic Module. You will have to specify a height when calling engage().


In [9]:
p20_sin = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tiprack_20_sin])
p300_sin = protocol.load_instrument("p300_single_gen2", "left", tip_racks=[tiprack_300_sin])

In [14]:
## Start Tiprack positions
p20_sin.starting_tip = tiprack_20_sin.well("A3")
p300_sin.starting_tip = tiprack_300_sin.well("F3")

In [15]:
## Sample Wells
start_position = 0 ## start from A1
sample_number = 2

In [16]:
dna_well = [dna_plate.wells()[i] for i in range(start_position, start_position + sample_number)]
final_well = [dna_plate.wells()[i] for i in range(start_position + 16, start_position + 16 + sample_number)]
mag_well = [mag_plate.wells()[i] for i in range(start_position, start_position + sample_number)]

## Reagents

In [17]:
ligation_buf = enzyme_rack["A1"]
t4_ligase = enzyme_rack["A2"]
adapter_II = enzyme_rack["A3"]

water = enzyme_rack["A5"]

In [160]:
ampure_beads = falcon_rack["A1"]
ethanol_70 = falcon_rack["A2"]

In [107]:
def enzyme_transfer(pipette, volume, src, dest, delay_second, asp_rate=None, dis_rate=None):
    if asp_rate != None:
        pipette.flow_rate.aspirate=asp_rate
    if dis_rate != None:
        pipette.flow_rate.dispense=dis_rate
    pipette.pick_up_tip()
    pipette.aspirate(volume, src)
    protocol.delay(seconds=delay_second)
    pipette.touch_tip()
    pipette.dispense(volume, dest)
    protocol.delay(seconds=delay_second/2)
    pipette.blow_out()    
    pipette.touch_tip()
    pipette.drop_tip()

In [ ]:
"""
LNB position = (z=-19) #top 기준
AMX z = -18 #top 기준
"""

In [109]:
enzyme_transfer(p300_sin, 25, 
                ligation_buf.top().move(types.Point(-2,1,-19)),
               dna_well[0], 1, asp_rate=25, dis_rate=20)

In [ ]:
enzyme_transfer(p300_sin, 10, 
                t4_ligase,
               dna_well[0], 1, asp_rate=10, dis_rate=10)

In [ ]:
enzyme_transfer(p300_sin, 5, 
                adapter_II.top().move(types.Point(-1,1,-18)),
               dna_well[0], 1, asp_rate=10, dis_rate=10)

In [165]:
p300_sin.flow_rate.aspirate=150
p300_sin.flow_rate.dispense=150
p300_sin.pick_up_tip()
p300_sin.mix(5, 300, ampure_beads.bottom(z=3))
protocol.delay(seconds=3)
p300_sin.blow_out()
p300_sin.touch_tip()
protocol.delay(seconds=3)
p300_sin.drop_tip()

<InstrumentContext: p300_single_v2.1 in LEFT>

In [171]:
# 옮기다 bead가 떨어지는 문제발생 = blowout 으로 해결
p300_sin.flow_rate.aspirate=25
p300_sin.flow_rate.dispense=25
p300_sin.transfer(50, 
                  ampure_beads.bottom(z=15), 
                  dna_well[0], mix_after=(3, 30), 
                  touch_tip = True, blow_out = True, 
                  blowout_location='destination well', new_tip="always")

<InstrumentContext: p300_single_v2.1 in LEFT>

In [174]:
module_magnetic.engage(height_from_base=3)
#protocol.delay(minutes=5)

In [194]:
## Functions
def remove_supernantant(pipette, volume, src, dest):
    pipette.pick_up_tip()
    pipette.flow_rate.aspirate=20
    pipette.aspirate(volume, src.bottom().move(types.Point(x=-1, y=0, z=-4.1)))
    pipette.dispense(volume, dest)
    pipette.drop_tip()

In [176]:
for src in mag_well:
    remove_supernantant(p300_sin, 120, src, trash)

In [198]:
p300_sin.flow_rate.aspirate=100
p300_sin.flow_rate.dispense=50
for _ in range(2):
    ## Ethanol
    p300_sin.pick_up_tip()
    for src in mag_well:
        p300_sin.flow_rate.aspirate=100
        p300_sin.flow_rate.dispense=50
        p300_sin.aspirate(190, ethanol_70)
        p300_sin.dispense(190, src.top(-3))
    p300_sin.drop_tip()
    ## Remove
    for src in mag_well:
        remove_supernantant(p300_sin, 200, src, trash)

In [199]:
module_magnetic.disengage()

In [155]:
p300_sin.flow_rate.asiprate=10
p300_sin.flow_rate.dispense=10
p300_sin.pick_up_tip()

<InstrumentContext: p300_single_v2.1 in LEFT>

In [156]:
p300_sin.aspirate(5, adapter_II.top().move(types.Point(-1,1,-18)))
protocol.delay(seconds=1)
p300_sin.touch_tip()

<InstrumentContext: p300_single_v2.1 in LEFT>

In [157]:
p300_sin.dispense(5, dna_well[1])
protocol.delay(seconds=0.5)
p300_sin.blow_out()
p300_sin.touch_tip()
p300_sin.move_to(dna_well[1].top())

<InstrumentContext: p300_single_v2.1 in LEFT>

In [196]:
p300_sin.drop_tip()

<InstrumentContext: p300_single_v2.1 in LEFT>

In [58]:
p300_sin.move_to(ligation_buf.top())


<InstrumentContext: p300_single_v2.1 in LEFT>

In [50]:
# Nanopore tube position [ z = -19 ]
p300_sin.move_to(ligation_buf.top().move(types.Point(-2,1,-19)))


<InstrumentContext: p300_single_v2.1 in LEFT>

In [48]:
p300_sin.transfer(25, ligation_buf.top().move(types.Point(-2,1,-19)),
                  dna_well, touch_tip = True, new_tip="always")


KeyboardInterrupt: 

In [ ]:
## Enzyme reaction
p20_sin.flow_rate.aspirate=5
p20_sin.flow_rate.aspirate=5
p300_sin.flow_rate.aspirate=25
p300_sin.flow_rate.aspirate=25

p300_sin.transfer(25, ligation_buf, dna_well, touch_tip = True, new_tip="always")
#p20_sin.transfer(10, t4_ligase, dna_well, touch_tip = True, new_tip="always", mix_after=(2,15))
#p20_sin.transfer(5, adapter_II, dna_well, touch_tip = True, new_tip="always")

In [13]:
protocol.delay(minutes=9)
protocol.pause("Move tubes to mag_plate and Resume")

TypeError: got an unexpected keyword argument 'm'

In [ ]:
## Functions
def remove_supernantant(pipette, volume, src, dest):
    pipette.pick_up_tip()
    pipette.flow_rate.aspirate=20
    pipette.aspirate(volume, src.bottom().move(types.Point(x=0, y=0, z=0)))
    pipette.dispense(volume, dest)
    pipette.drop_tip()